A atividade consiste na execução de algoritmos de classificação usando os valores dos hiperparâmetros padrão (use cross_val_score) e também realizando uma busca em grade (use GridSearchCV) visando encontrar os melhores valores para cada caso. A medida utilizada para avaliar os modelos deve ser a acurácia.

In [1]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# 1. Carregar o dataset
try:
    df = pd.read_csv('voice.csv')
except FileNotFoundError:
    print("Erro: O arquivo 'voice.csv' não foi encontrado. Verifique se o arquivo está no mesmo diretório do script.")
    exit()


In [3]:
# 2. Pré-processar os dados
X = df.drop('label', axis=1)
y = df['label']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [4]:
# 3. Definir o objeto de validação cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=0)

In [5]:
# 4. Definir algoritmos e grade de parâmetros
algorithms = {
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 4, 5, 6, 7, 8, 9],
            'weights': ['uniform', 'distance'],
            'metric': ['cityblock', 'euclidean']
        }
    },
    'SVM': {
        'model': SVC(random_state=0),
        'params': {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'gamma': [0.001, 0.01, 0.1, 1, 10, 100]
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=0),
        'params': {
            'max_depth': [None, 5, 7, 9],
            'min_samples_split': [2, 5, 10, 20, 30],
            'criterion': ['gini', 'entropy']
        }
    },
    'Logistic Regression': {
        'model': LogisticRegression(random_state=0, max_iter=3000),
        'params': {
            'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
        }
    }
}

In [6]:

# 5. Armazenar os resultados
results = {}

# 6. Loop para treinar e avaliar os modelos
for name, algo_info in algorithms.items():
    print(f"\n--- Processando {name} ---")
    model = algo_info['model']
    params = algo_info['params']

    # Avaliação com hiperparâmetros padrão
    print("Avaliando com parâmetros padrão...")
    default_scores = cross_val_score(model, X, y_encoded, cv=cv, scoring='accuracy')
    default_mean_accuracy = default_scores.mean()

    # Busca em grade para tuning
    print("Realizando Busca em Grade...")
    grid_search = GridSearchCV(model, params, cv=cv, scoring='accuracy')
    grid_search.fit(X, y_encoded)
    tuned_mean_accuracy = grid_search.best_score_
    best_params = grid_search.best_params_

    # Armazena os resultados
    results[name] = {
        'default_accuracy': default_mean_accuracy,
        'tuned_accuracy': tuned_mean_accuracy,
        'best_params': best_params
    }


--- Processando KNN ---
Avaliando com parâmetros padrão...
Realizando Busca em Grade...

--- Processando SVM ---
Avaliando com parâmetros padrão...
Realizando Busca em Grade...

--- Processando Decision Tree ---
Avaliando com parâmetros padrão...
Realizando Busca em Grade...

--- Processando Logistic Regression ---
Avaliando com parâmetros padrão...
Realizando Busca em Grade...


In [7]:
# 7. Exibir os resultados finais
print("\n" + "="*50)
print("Resumo dos Resultados Finais")
print("="*50)

summary_df = pd.DataFrame(results).T
summary_df['accuracy_diff'] = summary_df['tuned_accuracy'] - summary_df['default_accuracy']
print(summary_df)

# 8. Responder às questões
print("\n" + "-"*50)
print("Respostas para as Questões")
print("-"*50)

best_default = summary_df['default_accuracy'].idxmax()
best_tuned = summary_df['tuned_accuracy'].idxmax()
print(f"1. Melhor desempenho com parâmetros padrão: {best_default} com acurácia de {summary_df.loc[best_default, 'default_accuracy']:.4f}")
print(f"   Melhor desempenho com tuning: {best_tuned} com acurácia de {summary_df.loc[best_tuned, 'tuned_accuracy']:.4f}")

largest_diff_algo = summary_df['accuracy_diff'].idxmax()
largest_diff_value = summary_df['accuracy_diff'].max()
print(f"2. Maior diferença de desempenho: {largest_diff_algo} com uma diferença de {largest_diff_value:.4f}")

no_diff_algo = summary_df[summary_df['accuracy_diff'] < 1e-4].index
print(f"3. Algoritmos com diferença insignificante: {list(no_diff_algo)}")

svm_best_params = results['SVM']['best_params']
print(f"4. Melhores valores para SVM: C={svm_best_params['C']}, Gamma={svm_best_params['gamma']}")

worst_tuned = summary_df['tuned_accuracy'].idxmin()
worst_tuned_accuracy = summary_df.loc[worst_tuned, 'tuned_accuracy']
print(f"5. Pior desempenho após tuning: {worst_tuned} com acurácia de {worst_tuned_accuracy:.4f}")


Resumo dos Resultados Finais
                    default_accuracy tuned_accuracy  \
KNN                         0.715273       0.803332   
SVM                         0.672656       0.917293   
Decision Tree               0.968437       0.968437   
Logistic Regression         0.907509       0.971276   

                                                           best_params  \
KNN                  {'metric': 'cityblock', 'n_neighbors': 6, 'wei...   
SVM                                          {'C': 100, 'gamma': 0.01}   
Decision Tree        {'criterion': 'gini', 'max_depth': None, 'min_...   
Logistic Regression                                       {'C': 10000}   

                    accuracy_diff  
KNN                       0.08806  
SVM                      0.244637  
Decision Tree                 0.0  
Logistic Regression      0.063767  

--------------------------------------------------
Respostas para as Questões
--------------------------------------------------
1. Melhor des